In [17]:
with open('log1223_14.txt', 'r') as f:
    all_logs = f.readlines() 

In [18]:
import re

dataset_list = []
lr_list = []
epochs_list = []
hidden_list = []
dropout_list = []
weight_decay_list = []
alpha_list = []
beta_list = []
gamma_list = []
norm_layers_list = []
orders_list = []
highest_train_list = []
highest_valid_list = []
final_train_list = []
final_test_list = []
delta_list = []

for logs in all_logs:
    if 'Namespace' in logs:
        dataset_list.extend(re.findall(r"dataset='(.*?)', delta", logs))
        delta_list.extend(re.findall(r"delta=(.*?), directed", logs))
        lr_list.extend(re.findall(r"lr=(.*?), method", logs))
        epochs_list.extend(re.findall(r"epochs=(.*?), gamma", logs))
        dropout_list.extend(re.findall(r"dropout=(.*?), epochs", logs))
        weight_decay_list.extend(re.findall(r"weight_decay=(.*?)\)", logs))
        alpha_list.extend(re.findall(r"alpha=(.*?), beta", logs))
        beta_list.extend(re.findall(r"beta=(.*?), cached", logs))
        orders_list.extend(re.findall(r"orders=(.*?), orders_func_id", logs))
        gamma_list.extend(re.findall(r"gamma=(.*?), gat_heads", logs))
        norm_layers_list.extend(re.findall(r"norm_layers=(.*?), num_layers", logs))
        hidden_list.extend(re.findall(r"hidden_channels=(.*?), hops", logs))
        
    if 'Highest Train' in logs and ',' in logs:
        highest_train_list.extend(re.findall(r"Highest Train: (.*?),", logs))
    
    if 'Highest Valid' in logs and ',' in logs:
        highest_valid_list.extend(re.findall(r"Highest Valid: (.*?),", logs))
        
    if 'Final Train' in logs and ',' in logs:
        final_train_list.extend(re.findall(r"Final Train: (.*?),", logs))
        
    if 'Final Test' in logs and ',' in logs:
        final_test_list.extend(re.findall(r"Final Test: (.*?),", logs))


while len(highest_train_list) != len(dataset_list):
    highest_train_list.append('0')
    highest_valid_list.append('0')
    final_train_list.append('0')
    final_test_list.append('0')
        

In [19]:
import pandas as pd
import os

d = {
    'dataset': dataset_list,
    'lr': lr_list,
    'epoch': epochs_list,
    'hidden': hidden_list,
    'dropout': dropout_list,
    'weight_decay': weight_decay_list,
    'alpha': alpha_list,
    'beta': beta_list,
    'gamma': gamma_list,
    'delta': delta_list,
    'norm_layers': norm_layers_list,
    'orders': orders_list,
    'high_train': highest_train_list,
    'high_valid': highest_valid_list,
    'final_train': final_train_list,
    'final_test': final_test_list
}

for k, v in d.items():
    print(k, len(v))

csv_name = 'results/large_dataset_tuning14.csv'
df = pd.DataFrame.from_dict(d)
df = df.sort_values(['final_test']).reset_index(drop=True)
# if os.path.exists(csv_name):
#     df.to_csv(csv_name, mode='a', header=False)
# else:
#     df.to_csv(csv_name)

dataset 29
lr 29
epoch 29
hidden 29
dropout 29
weight_decay 29
alpha 29
beta 29
gamma 29
delta 29
norm_layers 29
orders 29
high_train 29
high_valid 29
final_train 29
final_test 29


In [20]:
df[-20:]

,dataset,lr,epoch,hidden,dropout,weight_decay,alpha,beta,gamma,delta,norm_layers,orders,high_train,high_valid,final_train,final_test
9,genius,0.001,500,256,0.0,0.0,0.0,1.0,0.8,0.5,2,1,94.10,90.59,93.48,90.59
10,genius,0.001,500,256,0.0,0.0,1.0,1.0,0.8,0.5,2,1,94.23,90.61,93.56,90.61
11,genius,0.001,500,256,0.0,1e-07,1.0,1.0,0.9,0.5,2,3,94.43,90.60,93.56,90.61
12,genius,0.001,500,256,0.0,0.0,0.0,1.0,0.9,0.5,2,1,94.35,90.62,93.88,90.61
13,genius,0.001,500,256,0.0,0.0,0.1,1.0,0.9,0.5,2,3,94.33,90.62,93.34,90.64
14,genius,0.001,500,256,0.0,0.0,0.0,1.0,0.8,0.5,2,3,94.70,90.65,94.07,90.66
15,genius,0.001,500,256,0.0,1e-07,0.0,1.0,0.9,0.5,2,2,94.66,90.68,94.11,90.69
16,genius,0.001,500,256,0.0,1e-07,0.0,1.0,0.9,0.5,2,3,94.91,90.71,94.26,90.74
17,genius,0.001,500,256,0.0,1e-07,0.0,1.0,0.9,0.5,2,1,94.77,90.74,94.31,90.74
18,genius,0.001,500,256,0.0,1e-07,1.0,1.0,0.9,0.5,2,1,94.16,90.79,93.75,90.75
